### This Notebook is created for completing the Capstone Project in IBM Data Science course on Coursera

In [1]:
import numpy as np
import pandas as pd
print ("Hello IBM Data Science Capstone Project Course!")

Hello IBM Data Science Capstone Project Course!


<a href="https://cognitiveclass.ai"><img src = "https://2u2berbm7s71ernxx1ssaws2-wpengine.netdna-ssl.com/wp-content/uploads/2013/03/segmentation_map.svg_-1080x675.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City of Canada</font></h1>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Scrape and Explore Dataset from Wikipedia</a>

2. <a href="#item2">Data Preprocessing and Cleaning</a>
    
3. <a href="#item2">Importing Geo Location Data</a>
    
4. <a href="#item2">Exploring & Clustering Neighborhoods</a>

Downloading all the dependencies

In [2]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [3]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


vincent-0.4.4        | 28 KB     | ##################################### | 1

In [4]:
!pip install wikipedia

  Stored in directory: /home/dsxuser/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library

print('Libraries imported.')

import urllib
import urllib.request
from bs4 import BeautifulSoup
import re
import wikipedia as wp
print('Wikipedia module imported.')

Libraries imported.
Wikipedia module imported.


## 1. Scrape and Explore Dataset from Wikipedia

In [6]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

The table that we want to scrape exists starting with an HTML "table" tag with a class identifier of ”wikitable sortable”.The rows start and end with "tr" and "tr" tags.The top row of headers has "th" tags while the data rows beneath for each postal code has "td" tags.It’s in these tags that we will tell Python to extract our data from.

In [113]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables=soup.find_all("table")

Looking through the output of ”all_tables” we can again see that the class id of our chosen table is ”wikitable sortable”. We can use this to get BS to only bring back the table data for this particular table and keep that in a variable called ”right_table“:

In [114]:
right_table=soup.find('table', class_='wikitable sortable')

Now we have to start looping through the rows to get the data for every Postal Code in the table.There are three columns in our table that we want to scrape the data from so we will set up five empty lists (A, B, C) to store our data in.To start with, we want to use the Beautiful Soup ‘find_all’ function again and set it to look for the string ‘tr’. We will then set up a FOR loop for each row within that array and set Python to loop through the rows, one by one.Within the loop we are going to use find_all again to search each row for "td" tags with the ‘td’ string. We will add all of these to a variable called ‘cells’ and then check to make sure that there are 3 items in our ‘cells’ array (i.e. one for each column).
If there are then we use the find(text=True)) option to extract the content string from within each "td" element in that row and add them to the A-C lists we created at the start of this step.

In [10]:
A=[]
B=[]
C=[]
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

We’ll now create a dataframe with it, assigning each of the lists A-C into a column with the name of our source table columns i.e.Postal Code,Borough & Neighborhood

In [11]:
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


This completes the second task of our assignment i.e Scraping the data from Wikipedia Page

## 2.Data Preprocessing and Cleaning

We will now process only those the cells that have an assigned borough and ignore those cells where borough is "Not assigned".

In [82]:
# Dropping the rows where Borough is 'Not assigned'
borough_drop = df[df['Borough'].apply(lambda x: x.startswith('Not assigned'))].index
df1 = df.drop(borough_drop)
df1

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [83]:
# Checking whether a cell has a borough but a Not assigned neighborhood
"Not assigned" in df1["Neighborhood"].values


False

Thus it is confimred that for every borough defined in the dataset a Neighborhood is also defined

In [84]:
# Combining the neighborhoods with same Postalcode with seperator "/" replaced by ","
df1["Neighborhood"]= df1["Neighborhood"].str.replace("/", ",", case = False)
df1["Neighborhood"]=df1["Neighborhood"].str.replace('\\n','') #Removing "\n" at the end of the line in each row
df1=df1.sort_values('PostalCode')
df1.reset_index(inplace=True,drop=True)
df1

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park"
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge"
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


In [85]:
# Shape of data frame
df1.shape

(103, 3)

This completes the task of transforming the data in the table on the Wikipedia page into the desired pandas dataframe of our assignment.

## 3.Importing Geo Location Data
Now that we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name,we will import the geographical coordinates of each postal code from csv file kept at http://cocl.us/Geospatial_data in order to utilize the Foursquare location data

In [86]:
df2 = pd.read_csv('https://cocl.us/Geospatial_data')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the Latitudes and Longitudes in our df1 data frame for various neighbourhoods in Canada

In [87]:
df2.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df2=df2.sort_values('PostalCode')
df2

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [89]:
#Removing trailing spaces from PostalCode column as directly going for a merge outputs an empty data frame
df1.PostalCode = [c.strip() for c in df1.PostalCode]
df2.PostalCode = [c.strip() for c in df2.PostalCode] 

In [93]:
df3=pd.merge(df1, df2, on='PostalCode')
df3

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848


## 4.Exploring & Clustering Neighborhoods
The notebook from here includes the Clustering and the plotting of the neighbourhoods of Canada which contain Toronto in their Borough.

In [94]:
#Getting all the rows from the data frame which contains Toronto in their Borough.
df4 = df3[df3['Borough'].str.contains('Toronto',regex=False)]
df4

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park , Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Summerhill West , Rathnelly , South Hill , For...",43.686412,-79.400049


Visualizing all the Neighbourhoods of the above data frame using Folium

In [102]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighborhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='cyan',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

Using KMeans clustering for the clustering of the neighbourhoods

In [108]:
# set number of clusters
k=5
toronto_clustering = df4.drop(['PostalCode','Borough','Neighborhood'],1)
# run k-means clustering
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 3, 3, 3, 3, 3, 3], dtype=int32)

In [110]:
df4

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,0,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,0,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
42,0,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572
43,0,M4M,East Toronto,Studio District,43.659526,-79.340923
44,1,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,1,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,1,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,1,M4T,Central Toronto,"Moore Park , Summerhill East",43.689574,-79.383160
49,1,M4V,Central Toronto,"Summerhill West , Rathnelly , South Hill , For...",43.686412,-79.400049


In [112]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighborhood, cluster in zip(df4['Latitude'], df4['Longitude'], df4['Neighborhood'], df4['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters